# Submiting Jobs to the C3 Cluster

## Running Asynchronos Batches of Functions

In order to take advanatage of more computational resources, one should employ one of the following mechinisms to submit long-running or embarassingly parallel batches:

    AsyncAction - Run a batch of jobs. Monitor and retrive results form the AsyncAction type and/or the FileSystem.
    BatchJob or DynBatchJob - More sophistaced batch jobs, persist results to your own type and/or the FileSystem
    MapReduce - For parallel jobs the complexity, as with other parallel compute systems, is wrangling the results. C3 uses a "type" system that can persist results to a database as well as a clter-wide "external" fileSystem (Azure Blob for DTI clusters).
    
Each of these approaches will put jobs in one of the cluster's queue's for execution on worker nodes.

The method we reccomend for dipatching execution of a python function to worker nodes for parallel execution without provisioning types is `AsyncAction`



### Submit a Single AsyncAction
Define a function and C3 "Lambda"

#### Define a function and its C3 "Lambda" version
Use the C3 type Lambda and its fromPython method to define a wrapped funtion that can be executed on the nodes of the C3 cluster.

In [1]:
def compute_pi(n):
    import decimal
    decimal.getcontext().prec = n + 1
    C = 426880 * decimal.Decimal(10005).sqrt()
    K = 6.
    M = 1.
    X = 1
    L = 13591409
    S = L

    for i in range(1, n):
        M = M * (K ** 3 - 16 * K) / ((i + 1) ** 3)
        L += 545140134
        X *= -262537412640768000
        S += decimal.Decimal(M * L) / X

        pi = C / S
    return str(pi)
compute_pi_c3 = c3.Lambda.fromPython(compute_pi)

def sleeper(n, say_something, something):
    import time
    time.sleep(n)
    if say_something:
        out = str(n) + '_' + something
    else:
        out = str(n)
    return out
sleeper_c3 = c3.Lambda.fromPython(sleeper)
# example execution locally
#sleeper_c3.apply([2])

#### Define the `AsyncActionSpec` for a Specific Run
Here, we are specifying how to call the `apply` method in the C3 `Lambda` type.  Which allows us to "apply" the Lambda we created from the function we wanted to execute.  
We pass "args" to the `apply` method which take the form of a list of inputs [input1, input2, ...].
the `c3.c3Make()` call serializes the native input list so that it plays nice with the API's on the C3 platform.

In [3]:
actionSpec = c3.AsyncActionSpec(
    typeName = "Lambda",
    action = "apply",
    args = {
        "this": sleeper_c3,
        "args": c3.c3Make("[any]",[2,True,"string"])
    }
)
actionSpec

c3.AsyncActionSpec(
 typeName='Lambda',
 actionName='null',
 args=c3.Mapp<string, any>({'args': c3.Arry<any>([2, True, 'string']),
        'this': c3.Lambda<function(n: any, say_something: any, something: any): any>(
                 language='python',
                 implementation='def sleeper(n, say_something, something):\n'
                                 '    import time\n'
                                 '    time.sleep(n)\n'
                                 '    if say_something:\n'
                                 "        out = str(n) + '_' + something\n"
                                 '    else:\n'
                                 '        out = str(n)\n'
                                 '    return out')}),
 action='apply')

#### Submit and Monitor the action

In [4]:
action = c3.AsyncAction.submit(actionSpec)

In [5]:
action = c3.AsyncAction.get({'id': action.id})
done = action.hasCompleted()
if done:
    print(action.result)

In [17]:
action

c3.AsyncAction(
 typeName='Lambda',
 actionName='apply',
 args=c3.Mapp<string, any>({'args': c3.Arry<any>([2, True, 'string']),
        'this': c3.Lambda<function(n: any, say_something: any, something: any): any>(
                 language='python',
                 implementation='def sleeper(n, say_something, something):\n'
                                 '    import time\n'
                                 '    time.sleep(n)\n'
                                 '    if say_something:\n'
                                 "        out = str(n) + '_' + something\n"
                                 '    else:\n'
                                 '        out = str(n)\n'
                                 '    return out')}),
 action='apply',
 id='7109ee47-d244-4bad-9fcd-fcee2a1a1e5d',
 meta=c3.Meta(
        tenantTagId=150,
        tenant='dev',
        tag='tc01d',
        created=datetime.datetime(2022, 2, 16, 20, 29, 40, tzinfo=datetime.timezone.utc),
        createdBy='dadams@illinois.e

### Submit Async Action to a Particular Hardware Profile

In [6]:
profiles = c3.HardwareProfile.fetch().objs
for p in profiles:
    print(p.name, p.instanceType)

GPU_V100_SMALL STANDARD_NC6S_V3


In [13]:
def gpuTest():
    import tensorflow as tf
    from tensorflow.python.client import device_lib
    devs = device_lib.list_local_devices()
    print(devs)
    return list(map(str,devs))

In [12]:
gpuTest()

ModuleNotFoundError: No module named 'tensorflow'

In [14]:
gpuTest_c3 =c3.Lambda.fromPython(gpuTest,runtime="tensorflow_3_0_0")

In [20]:
# hardwareProfile = "GPU_V100_SMALL"
actionSpec = c3.AsyncActionSpec(
    typeName = "Lambda",
    action = "apply",
    args = {
        "this": gpuTest_c3
    },
    hardwareProfile = "GPU_V100_SMALL"
)
actionSpec

c3.AsyncActionSpec(
 typeName='Lambda',
 actionName='null',
 args=c3.Mapp<string, any>({'this': c3.Lambda<function(): any>(
                 language='python',
                 implementation='def gpuTest():\n'
                                 '    import tensorflow as tf\n'
                                 '    from tensorflow.python.client import '
                                 'device_lib\n'
                                 '    devs = device_lib.list_local_devices()\n'
                                 '    print(devs)\n'
                                 '    return list(map(str,devs))',
                 runtime='py-tensorflow_3_0_0')}),
 action='apply',
 hardwareProfile='GPU_V100_SMALL')

In [21]:
action = c3.AsyncAction.submit(actionSpec)

In [24]:
action = c3.AsyncAction.get({'id': action.id})
done = action.hasCompleted()
if done:
    print(action.result)
else:
    print(action)

['name: "/device:CPU:0"\ndevice_type: "CPU"\nmemory_limit: 268435456\nlocality {\n}\nincarnation: 14264760189499237810\n', 'name: "/device:XLA_CPU:0"\ndevice_type: "XLA_CPU"\nmemory_limit: 17179869184\nlocality {\n}\nincarnation: 12365955161697616446\nphysical_device_desc: "device: XLA_CPU device"\n']


In [22]:
action

c3.AsyncAction(
 typeName='Lambda',
 actionName='apply',
 args=c3.Mapp<string, any>({'args': c3.Arry<any>([3000]),
        'this': c3.Lambda<function(n: any): any>(
                 language='python',
                 implementation='def compute_pi(n):\n'
                                 '    import decimal\n'
                                 '    decimal.getcontext().prec = n + 1\n'
                                 '    C = 426880 * '
                                 'decimal.Decimal(10005).sqrt()\n'
                                 '    K = 6.\n'
                                 '    M = 1.\n'
                                 '    X = 1\n'
                                 '    L = 13591409\n'
                                 '    S = L\n'
                                 '\n'
                                 '    for i in range(1, n):\n'
                                 '        M = M * (K ** 3 - 16 * K) / ((i + 1) '
                                 '** 3)\n'
                        

### Submit a Batch of `AsyncAction`s

In [19]:
import numpy as np
inputs = np.random.randint(100,120,100)
specs = [
    c3.AsyncActionSpec(
    typeName = "Lambda",
    action = "apply",
    args = {
        "this": sleeper_c3,
        "args": c3.c3Make("[int]",[n])
    }
) for n in inputs
]

In [20]:
for spec in specs:
    c3.AsyncAction.submit(spec)

In [27]:
c3.Type(**{
    "from": {"name": "SleeperInput"}
}
    )

TypeError: type.__new__() takes exactly 3 arguments (0 given)

In [28]:
c3._box() 

TypeError: _box() missing 1 required positional argument: 'obj'

In [7]:
for job in c3.AsyncAction.fetch(spec={"include":"id,completed"}).objs:
    #stat = job.status().status
    stat = job.completed
    print(stat)
#    if not stat:
#         print(stat)
#         #job.cancel()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [1]:
c3.AsyncAction.fetchCount()

1238